PACKAGES

In [1]:
from deepod.models.tabular import DevNet, PReNet, DeepSAD, FeaWAD, RoSAS
from deepod.metrics import tabular_metrics
from autoencodernn import *
from tapnet import *

/home/work/anaconda3/envs/dongjae_rba/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
import os
from datetime import datetime

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

DATASET

In [5]:
dfs = []
for filename in tqdm(os.listdir('./data/')):
    if 'preprocessed' in filename:
        dfs.append(pd.read_csv(f'./data/{filename}', index_col = 0))
df = pd.concat(dfs).reset_index(drop = True)
del(dfs)

100% 35/35 [00:11<00:00,  2.93it/s]


In [6]:
train_valid_df, test_df = train_test_split(df, test_size = 0.2, stratify = df['label'])
train_df, valid_df = train_test_split(train_valid_df, test_size = 0.125, stratify = train_valid_df['label'])

In [7]:
# X_train = train_df.drop(['label'], axis = 1)
# y_train = train_df['label']

# X_valid = valid_df.drop(['label'], axis = 1)
# y_valid = valid_df['label']

# X_test = test_df.drop(['label'], axis = 1)
# y_test = test_df['label']

In [ ]:
# X_train.to_csv('./checkpoint/x_train.csv')
# y_train.to_csv('./checkpoint/y_train.csv')

# X_valid.to_csv('./checkpoint/x_valid.csv')
# y_valid.to_csv('./checkpoint/y_valid.csv')

# X_test.to_csv('./checkpoint/x_test.csv')
# y_test.to_csv('./checkpoint/y_test.csv')

In [8]:
X_train = pd.read_csv('./checkpoint/x_train.csv')
y_train = pd.read_csv('./checkpoint/y_train.csv')

X_valid = pd.read_csv('./checkpoint/x_valid.csv')
y_valid = pd.read_csv('./checkpoint/y_valid.csv')

X_test = pd.read_csv('./checkpoint/x_test.csv')
y_test = pd.read_csv('./checkpoint/y_test.csv')

MODEL

In [9]:
models = []

In [10]:
def anomaly_preprocessing(df1, df2, df3):
    df1['type'] = 'train'
    df2['type'] = 'valid'
    df3['type'] = 'test'

    df = pd.concat([df1, df2, df3]).reset_index(drop = True)

    country_onehot = pd.get_dummies(df['country']).astype(int)
    risk_grades = df[['region_risk_grade', 'city_risk_grade', 'name_risk_grade']]
    browser_onehot = pd.get_dummies(df['browser_name']).astype(int)
    os_onehot = pd.get_dummies(df['os_name']).astype(int)
    legacys = df[['browser_is_legacy', 'os_is_legacy']]
    device_types = pd.get_dummies(df['device_type']).astype(int)
    rtts = df['rtt']
    type = df['type']
    label = df['label']
    df = pd.concat([country_onehot, risk_grades, browser_onehot, os_onehot, legacys, device_types, rtts, type, label], axis = 1)

    df1 = df[df['type'] == 'train'].drop('type', axis = 1)
    df2 = df[df['type'] == 'valid'].drop('type', axis = 1)
    df3 = df[df['type'] == 'test'].drop('type', axis = 1)

    return df1, df2, df3

In [ ]:
# Anomaly Detection
print('Anomaly Detection Model')
ad_model_names = ['DevNet', 'PReNet', 'DeepSAD', 'FeaWAD', 'RoSAS']
ad_models = [
    DevNet(),           # 86.5s
    # PReNet(),           # Too long. (2.4 hours)
    DeepSAD(),          # 75.4s
    # FeaWAD(epochs = 10000, lr = 0.01),           # Very fast but poor.
    # RoSAS(),            # Too long.
]

train_df_ad, valid_df_ad, test_df_ad = anomaly_preprocessing(train_df, valid_df, test_df)

X_train_ad = train_df_ad.drop(['label'], axis = 1)
y_train_ad = train_df_ad['label']

X_valid_ad = valid_df_ad.drop(['label'], axis = 1)
y_valid_ad = valid_df_ad['label']

X_test_ad = test_df_ad.drop(['label'], axis = 1)
y_test_ad = test_df_ad['label']

for model_name, model in zip(ad_model_names, ad_models):
    print('start -', datetime.now())
    
    model.fit(X_train_ad.to_numpy(), y_train_ad.to_numpy())
    print('Train Finish')
    pred_train = (model.decision_function(X_train_ad.to_numpy()) > 0.5).astype(int)
    auc_train, ap_train, f1_train = tabular_metrics(y_train_ad, pred_train)
    
    pred_valid = (model.decision_function(X_valid_ad.to_numpy()) > 0.5).astype(int)
    auc_valid, ap_valid, f1_valid = tabular_metrics(y_valid_ad, pred_valid)
    
    print(f'Trained with {model}')
    print(f'Train - AUC: {auc_train}, AP: {ap_train}, F1: {f1_train}')
    print(f'Valid - AUC: {auc_valid}, AP: {ap_valid}, F1: {f1_valid}')

    models.append(model)
    print('end -', datetime.now(), '\n')

Anomaly Detection Model
start - 2023-12-10 10:08:11.300161
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=220, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=1, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 2.497916, time: 101.3s
epoch 10, training loss: 2.500566, time: 99.9s
epoch 30, training loss: 2.500028, time: 99.5s
epoch 40, training loss: 2.494651, time: 99.9s
epoch 70, training loss: 2.491216, time: 100.0s
epoch 80, training loss: 2.489513, time: 99.9s
epoch 90, training loss: 2.489794, time: 99.3s
epoch100, training loss: 2.490551, time: 99.3s
Start Inference on the training data...


testing: 100% 16653/16653 [00:29<00:00, 568.46it/s]


Train Finish


testing: 100% 16653/16653 [00:29<00:00, 573.96it/s]
testing: 100% 2379/2379 [00:04<00:00, 581.21it/s]


Trained with <deepod.models.tabular.devnet.DevNet object at 0x7fad9d16fc10>
Train - AUC: 0.7447359488557014, AP: 0.32677451001144814, F1: 0.5345130108361602
Valid - AUC: 0.7446193461758034, AP: 0.32537918443120517, F1: 0.5332294333741512
end - 2023-12-10 12:53:31.695852 

start - 2023-12-10 12:53:31.695873
Start Training...
ensemble size: 1
training data counter: Counter({0: 971745, -1: 94036})
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=220, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 30.813188, time: 92.3s
epoch 10, training loss: 1.198376, time: 92.4s
epoch 20, training loss: 1.027989, time: 94.2s
epoch 30, training loss: 0.957365, time: 90.5s


In [ ]:
# AutoEncoder + NN
print('AutoEncoder + NN Model')
autoencoder_nn_model = model_v2(
    train_data = train_df,          # Train set
    valid_data = valid_df,          # Validation set
    test_data = test_df,            # Test set
    criteria = 0.5,                 # Classification threshold
    split_ratio = [7, 1, 2],        # split ratio (format: [train,validation,test])
    autoencoder_epochs = 50,        # epochs of autoencoder
    classifier_epochs = 200,        # epochs of classifier
    weight_for_attack = 15,         # weight for attack
)

models.append(autoencoder_nn_model)

In [ ]:
# TabNet
print('TabNet Model')
selected_columns = ['country_code', 'region', 'city_risk_grade', 'name_risk_grade', 'login_success', 'browser_is_legacy', 'os_is_legacy', 'rtt', 'device_type', 'label']
categorical_columns = ['country_code', 'device_type', 'region']

tabnet_model = TabNetModel(train_df, valid_df, test_df, selected_columns, categorical_columns, 'label')         #, pre_train_epochs = 5, epochs = 5)
models.append(tabnet_model)

In [ ]:
with open('./checkpoint/models.pkl', 'wb') as f:
    pickle.dump(models, f)